In [ ]:
import pandas as pd

from data_preparation import (
    download_gios_archive,
    split_raw_df_to_metadata_and_measurements,
    download_updated_metadata,
    build_station_code_mapping,
    update_station_names_metadata,
    update_station_names_data,
    extend_metadata_with_station_info,
    combine_metadata_frames,
    combine_data_frames,
)
from statistics_calculation import (
    analyze_raw_df,
    check_timestamps,
    report_nan_runs,
    monthly_avg_with_nan_threshold,
    average_by_city,
    count_days_over_threshold,
)
from visualizations import (
    plot_monthly_avg_station_per_year,
    plot_monthly_avg_station_mean_std_per_year,
    plot_monthly_avg_station_comparison,
    plot_city_monthly_averages,
    plot_city_monthly_heatmaps,
    plot_extreme_stations_days_over,
)

# Przygotowanie danych

## Pobieranie surowych danych

Ustawienia danych i metadanych:
- ścieżki do pobrania plików
- oczekiwane hashe dla odtwarzalności
- poszukiwane pliki w archiwach

In [ ]:
gios_archive_urls = {
    2014: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/302",
    2015: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/236",
    2018: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/603",
    2019: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/322",
    2021: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/486",
    2024: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/582",
}
gios_pm25_file = {
    2014: "2014_PM2.5_1g.xlsx",
    2015: "2015_PM25_1g.xlsx",
    2018: "2018_PM25_1g.xlsx",
    2019: "2019_PM25_1g.xlsx",
    2021: "2021_PM25_1g.xlsx",
    2024: "2024_PM25_1g.xlsx",
}
gios_archive_sha256 = {
    2014: "8cabcc2118f019d8d1c0998561c01d57eda8c0a4c531cd2158b18522cd1aed27",
    2015: None,
    2018: None,
    2019: "777bc03c3c6d1ac77bd4353a80b6e064506368d42be19edece60f040c17dba1c",
    2021: None,
    2024: "571dfa56866388c2904284ca6029bbf6016af3905b95bcacc5b3b6f6fa2d00e1",
}
gios_metadata_url = "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/622"
#gios_metadata_url = "https://web.archive.org/web/20251230110906/https://powietrze.gios.gov.pl/pjp/archives/downloadFile/622"
gios_metadata_sha256 = "174290b98ceb780c69769806f7f7a6054015cd78ead8ee65746f3fceba66b2ab"

Sprawdzam hashe (wpisane jednorazowo) dla pełnej odtwarzalności (reproducibility)

Funkcja jest w pełni parametryzowana - przekazujemy pełny URL archiwum, nazwę pliku w ZIP i opcjonalny hash.

In [ ]:
years = [2015, 2018, 2021, 2024]
raw_dfs = {
    year: download_gios_archive(
        gios_archive_urls[year],
        gios_pm25_file[year],
        sha256=gios_archive_sha256[year],
    )
    for year in years
}

Pobieramy najnowszy plik z metadanymi, w tym aktualizacją nazw stacji, z GIOŚ.

In [ ]:
updated_metadata_df = download_updated_metadata(
    gios_metadata_url,
    sha256=gios_metadata_sha256,
)

## Sprawdzenie pobranych danych

In [ ]:
meta_keys = ["Nr", "Kod stacji", "Wskaźnik",
             "Czas uśredniania", "Jednostka", "Czas pomiaru", "Kod stanowiska"]

for year in years:
    year_df = raw_dfs[year]
    print(f"Sanity check for raw {year} data:\n")
    print(analyze_raw_df(year_df, meta_keys))
    print(check_timestamps(year_df, meta_keys, 'h'))
    print(f"\n\n")

## Zmiana formatu dataframe'ów przed dalszym procesowaniem

Dla większej przejrzystości (i zgodnie z zasadami normalizacji) zapisujemy i formatujemy metadane i dane pomiarowe w osobnych DataFrame'ach.

Polecenie wspomina jedynie o *możliwości* skorzystania z MultiIndexu, ostatecznie postanowiliśmy wyraźnie oddzielić metadane stacji od danych pomiarowych.

In [ ]:
meta_keys = ["Nr", "Kod stacji", "Wskaźnik",
             "Czas uśredniania", "Jednostka", "Czas pomiaru", "Kod stanowiska"]

meta_by_year = {}
data_by_year = {}
for year in years:
    meta_by_year[year], data_by_year[year] = split_raw_df_to_metadata_and_measurements(raw_dfs[year], meta_keys)

Szybkie sprawdzenie jak wygląda frame danych.

In [ ]:
data_by_year[2015].head()

## Aktualizacja nazw stacji

Szybki rzut oka czy dane wyglądają poprawnie

In [ ]:
updated_metadata_df.head()

Budujemy słownik mapujący stare nazwy do nowych.

Tzn. indeksy zawierają wszystkie historyczne nazwy stacji (oraz obecne) a przypisane są im aktualne nazwy stacji.

Zatem:
- aktualne kody mapują się do siebie (tożsamość),
- stare kody mapują się do odpowiadających im nowych kodów.

Dzięki temu otrzymujemy uniwesalną mapę:

mapa_kodów [ jakaś nazwa stacji, może aktualna, może nie ] --> pewna, aktualna nazwa tej stacji

Być może ten sposób jest "bardzo skomplikowany", ale wydaje się logiczny i rozszerzalny wobec kolejnych zmian

In [ ]:
code_map = build_station_code_mapping(updated_metadata_df)

Aktualizujemy metadane i dane pomiarowe -> nowe, aktualne nazwy stacji

In [ ]:
meta_by_year_updated = {
    year: update_station_names_metadata(meta_by_year[year], updated_metadata_df, code_map, label=str(year))
    for year in years
}

In [ ]:
data_by_year_updated = {
    year: update_station_names_data(data_by_year[year], code_map, label=str(year))
    for year in years
}

## Uzupełnienie i rozszerzenie metadanych

Dodajemy wybrane kolumny z updated_metadata_df (pełne metadane GIOŚ) do naszego głównego Frame'a z metadanymi

W połączonej ramce nie brak informacji o miejscowości.

In [ ]:
extra_cols = [
    "Typ stacji",
    "Typ obszaru",
    "Rodzaj stacji",
    "Województwo",
    "Miejscowość",
    "WGS84 φ N",
    "WGS84 λ E",
]

meta_by_year_extended = {
    year: extend_metadata_with_station_info(meta_by_year_updated[year], updated_metadata_df, extra_cols, label=str(year))
    for year in years
}

Niewątpię, że taki format spełnia standardy World Geodetic System '84, jednak przemianowyjemy te kolumny, choćby po to, by unikać konieczności wstawiania greckich symboli. 

In [ ]:
rename_geo = {
    "WGS84 φ N": "Szerokość geograficzna",
    "WGS84 λ E": "Długość geograficzna",
}

meta_by_year_extended = {year: df.rename(columns=rename_geo) for year, df in meta_by_year_extended.items()}

## Połączenie danych dla różnych lat

Tworzymy połączony zbiór dla niniejszego opracowania

In [ ]:
metadata_combined, diag1 = combine_metadata_frames(list(meta_by_year_extended.values()))

diag1

In [ ]:
data_combined, diag2 = combine_data_frames(list(data_by_year_updated.values()))

diag2

Sprawdzamy czy dane diagnostyczne (zachowane stacje) zgadzają się w ramce pomiarów i metadanych

In [ ]:
assert not diag1.isdisjoint(diag2)

## Czyszczenie / ektrapolacja brakujących danych

In [ ]:
report = report_nan_runs(data_combined, top_k=3)

print("Total stations:", report["total_stations"])
print("Stations with any NaN:", report["stations_with_any_nan"])
print(f"\n")

for station in report["nan_runs"].keys():
    print(station)
    print(report["nan_runs"][station])
    print(f"\n")

**Interpretacja**

W kilku miastach zdażają się wielomiesięczne przerwy w pomiarach.

W pozostałych miastach zdarzają się przerwy rzędu kilkunastu dni.

Jako, że mamy ograniczoną ilość danych - nie wyrzucam tych miast, nie uzupełniam luk.

Natomiast musimy pamiętać:
- Dla analizy miesięcznej: zignorować miesiące z większością dni NaN - niewiarygodna średnia
- Dla analizy dni z przekroczeniami: zignorować miasta z największymi / najdłuższymi brakami - wielomiesięczne luki zaniżają liczbę przekroczeń

## Zapisanie kopii Dataframe'ów na dysku

Surowe dane

In [ ]:
for year in years:
    raw_dfs[year].to_csv(f"raw_data/raw_data_{year}.csv.xz")

updated_metadata_df.to_csv("raw_data/raw_updated_metadata_20251230.csv.xz") # Uwaga - nie są to idealnie surowe dane - skasowaliśmy trailing space w nazwie jednej stacji

Oczyszczone dane

In [ ]:
for year in years:
    meta_by_year_extended[year].to_csv(f"cleaned_data/metadata_{year}.csv.xz")

for year in years:
    data_by_year_updated[year].to_csv(f"cleaned_data/data_{year}.csv.xz")

metadata_combined.to_csv("cleaned_data/pd3_dataset_metadata.csv.xz", index=False)
data_combined.to_csv("cleaned_data/pd3_dataset_data.csv.xz")

# Analiza

## Wczytanie danych

In [ ]:
metadata_combined = pd.read_csv("cleaned_data/pd3_dataset_metadata.csv.xz")
data_combined = pd.read_csv("cleaned_data/pd3_dataset_data.csv.xz", index_col=0, low_memory=False)

In [ ]:
data_combined.head()

In [ ]:
metadata_combined.head()

## Średnie miesięczne stacji

> Oblicz średnie miesięczne stężenie PM2.5 dla każdej stacji i roku

Średnie miesięczne stężenie dla każdej stacji rozumiem.
Natomiast "i roku" nie rozumiem.

Chodzi o stężenia miesięczne i roczne?

Średnie stężenie w danym roku we wszystkich stacjach?

In [ ]:
monthly_avg_station = monthly_avg_with_nan_threshold(data_combined, max_nan_per_month=24*10)

In [ ]:
monthly_avg_station.head()

In [ ]:
  plot_monthly_avg_station_per_year(
      monthly_avg_station,
      main_title="Średnie miesięczne dla stacji",
      main_title_y=0.95,
      main_title_fontsize=18,
      ncols=1,
      panel_size=(10, 4),
      hspace=0.30,
      wspace=0.0,
      label_fontsize=12,
      right_margin=0.99,
      legend_fontsize="large",
      legend_title_fontsize=15,
      sharey=True,
      sharex=False,
      marker_size=5
  )


Wykresy wskazują na silną sezonowość stężenia drobnego pyłu. Najwyższe wartości konsekwentnie przypadają na miesiące zimowe (styczeń-marzec i listopad-grudzień). Natomiast najniższe stężenia obserwowane są późną wiosną i latem. Tendencja ta utrzymuje się w skali całego kraju i jest spójnie zachowana w znakomitej większości stacji. Sugeruje to silny wpływ sezony grzewczego i warunków meteorologicznych na stężenie pyłu zawieszonego.

Na przestrzeni kolejnych lat widać wyraźny spadek stężeń pyłów, ponadto wydeje się, że rozrzut między stacjami staje się mniejszy. Może to wskazywać zarówno na poprawę jakości powietrza, częstsze wykorzystanie nowoczesnych, niskoemisyjnych metod grzewczych, ale może być również wynikiem łągodniejszych zim.

In [ ]:
plot_monthly_avg_station_mean_std_per_year(
    monthly_avg_station,                                    
    main_title="Średnie miesięczne połączonych stacji + std dev",
    main_title_y=0.95,
    main_title_fontsize=18,
)


Ten wykres pokazuje uśrednione wartości PM2.5 dla wszystkich stacji oraz pasmo +/-1 odchylenia standardowego. Sezonowość jest bardzo czytelna: zimą średnie są najwyższe i jednocześnie obserwujemy największą zmienność (szerokie pasma), co sugeruje silne zróżnicowanie lokalne i wrażliwość na epizody smogowe.

W 2024 roku zarówno średni poziom, jak i zmienność są zauważalnie niższe niż w 2015–2018. To wzmacnia tezę o poprawie jakości powietrza i stabilizacji stężeń w późniejszych latach.

In [ ]:
plot_monthly_avg_station_comparison(monthly_avg_station)

Zestawienie lat na jednym wykresie uwidacznia trend spadkowy: 2015 i 2018 mają najwyższe wartości w zimie, natomiast 2021, a szczególnie 2024, są wyraźnie niżej. Różnice są największe w miesiącach zimowych, co dodatkowo wspiera tezę, że zmiany (np. w ogrzewnictwie) są głóœną przyczyną poprawy jakości powietrza.

Latem krzywe zbliżają się do siebie, co oznacza, że poza sezonem grzewczym stężenia są bardziej stabilne i mniej zmienne w kolejnych latach. To dodatkowo wskazuje, że obserwowany spadek wynika głównie z redukcji zimowych epizodów smogowych.

## Warszawa vs Katowice

In [ ]:
city_data_combined = average_by_city(data_combined, metadata_combined)
city_data_combined.head()

In [ ]:
monthly_avg_city = monthly_avg_with_nan_threshold(city_data_combined, max_nan_per_month=24*10)

In [ ]:
monthly_avg_city

In [ ]:
plot_city_monthly_averages(monthly_avg_city, ["Warszawa", "Katowice"], [2015, 2024])

Porównanie Warszawy i Katowic pokazuje konsekwentnie wyższe poziomy PM2.5 w Katowicach, szczególnie zimą. Może to wynikać z bardziej przemysłowego z profilu regionu oraz gęstości zabudowy (konurbacja górnośląska). W obu miastach widoczny jest wyraźny spadek w 2024 roku względem 2015, zwłaszcza w miesiącach zimowych.

Różnice sezonowe są nadal silne, ale amplituda wahań jest wyraźnie mniejsza w 2024 r. Sugeruje to, że poprawa jakości powietrza jest trwała, choć Katowice wciąż pozostają bardziej narażone na wysokie stężenia niż Warszawa.

## Heatmapy

In [ ]:
plot_city_monthly_heatmaps(
    monthly_avg_city,
    cities=city_data_combined.columns.to_list(),
    years=[2015, 2018, 2021, 2024],
    ncols=2,
)

Mapy ciepła pokazują rozkład średnich miesięcznych PM2.5 w ujęciu miasto–rok. Na niemal każdym panelu widać cieplejsze kolory w miesiącach zimowych, co potwierdza silną sezonowość. Ponadto, różnice pomiędzy miastami są zauważalne: część ośrodków ma konsekwentnie wyższe wartości przez większość zimy.

Widoczny jest także ogólny trend obniżania się stężeń w kolejnych latach, zwłaszcza w 2024 roku. Sugeruje to poprawę jakości powietrza w skali całęgo kraju, z utrzymywaniem się lokalnych gorących punktów.

## Przekroczenia norm

In [ ]:
EXCLUDE_STATIONS = ["MzSiedKonars", "PmGdaLeczkow"] # wielomiesięczne przerwy w danych
days_over = count_days_over_threshold(data_combined.drop(EXCLUDE_STATIONS, axis=1), 15, years=(2015, 2018, 2021, 2024))
days_over

In [ ]:
plot_extreme_stations_days_over(
    days_over,
    year_ref=2024,
    years=(2015, 2018, 2021,2024),
    n=3,
    station_metadata=metadata_combined,
)


Wykres pokazuje, zmiany liczby dni z przekroczeniem normy dobowej w czasie oraz różnicę pomiędzy stacjami (najlepszymi i najgorszymi w 2024). Widać wyraźny spadek liczby przekroczeń w 2024 roku względem 2015 i 2018, co jest spójne z trendem obniżania średnich stężeń.

Jednocześnie rozrzut pomiędzy stacjami pozostaje duży: niektóre lokalizacje nawet w 2024 roku nadal notują istotną liczbę przekroczeń. To wskazuje, że mimo ogólnej poprawy, jakość powietrza jest wciąż silnie zróżnicowana lokalnie.

## Podsumowanie i Interpretacja

Na podstawie danych można stwierdzić, że sytuacja w zakresie stężenia pyłów **PM2.5** się poprawia na przestrzeni lat. Zdecydowanie widać pogorszenie sytuacji w okresie grzewczym, związanym z emisją pyłów przez prywatne ogrzewanie, które wedle mojej wiedzy, jest wysoko-emisje i przede-wszystkim odpowiedzialne za tak zwany smog. Widać tutaj skutek dotacji mających na celu wymienianie pieców cieplnych na nowoczesne.

Dodatkowo, sytuacja jest różna w różnych badanych regionach. Wynika to prawdopodobnie z czynników socjo-ekonomicznych (koszt wymiany pieca, ogrzewanie centralne, brak poczucia potrzeby dbania o środowisko), a także z uwarunkowania geograficznego (tj. faktu że niektóre miejscowości, są *uwięzione* przez uwarunkowanie terenowe, w taki sposób że zimą, podczas zwiększonej emisji, nie ma na tyle silnego wiatru, bądź jego kierunek nie pozwala na *wywianie* zanieczyszczeń)

### Suchy opis

- Progresywny spadek stężeń w kolejnych latach obserwowany w całym kraju - prawdopodobnie efekt zmian w technologiach grzewczych, spadek udziału paliw stałych (+ możliwe, że również łagodniejsze zimy)
- Silny efekt sezonowości - wzrost w miesiącach jesienno-zimowych - Prawdopodobnie wpływ "niskiej emisji" w sezonie grzewczym
- Wysokie zróżnicowanie stacji w miesiącach jesienno-zimowych, latem spadek we wszystkich stacjach do podobnego poziomu - Prawdopodobnie efekty zróżnicowanych lokalnie metod grzewczych, zróżnicowany udział paliw stałych i pieców starszej generacji
- Wspólny punkt odniesienia latem - prawdopodobnie efekt niższego zróżnicowania stałych źródeł zanieczyszczeń, głównie transportu
- Zróżnicowanie geograficzne: mniejsze poziomy w miejscowościach nadmorskich i na północy kraju (Gdańsk, Szczecin), wyróżniające się negatywnie miasta południa kraju (Kraków, Katowice)
- Ostatni wykres z przekroczeniami dziennymi również wskazuje że spadek stężeń występuje zarówno w "najlepszych" jak i "najgorszych" stacjach

## Zadanie 5

### Metodologia
Naszym celem jest określenie ilości dni które przekraczają norme ($15\mu g/m^3$). Uznawaliśmy, że dzień przekroczył normę, jeśli dowolna stacja w województwie przekroczyła próg.

### Wyniki
Widzimy, że w roku 2015, ilość przekroczonych dni wynosi średnio $\approx 250$, w niektórych województwach jest to nawet $300 >$ dni. 

W latach 2021 i 2024, spadek dni z przekroczeniem normy wynosi średnio $\approx 150$ , najgorszej sprawa się ma w województwie Lubelskim, Lubuskim, Podkarpackim, Małopolskim i Wielkopolskim

### Wnioski
Widzimy że Uchwały antysmogowe oraz program "Czyste Powietrze" przynoszą skutki jeżeli chodzi  poprawe sytuacji. Możemy podejrzewać, że przekroczenia wynikają głównie z powodu nieprawidłowego ogrzewania mieszkań. Żeby to sprawdzić trzeba by podzielić dane na dwie grupy - sezon grzewczy i pozostałe. 



Nie możemy określic dokładnych przyczyn z powodu dużego zróżnicowania umiejscowienia stacji. 

Ważne jest zwrócenie uwagi na problem metodologiczny, ponieważ w przypadku województw z dużą ilością stacji pomiarowych, częściej otrzymamy pozytywny wynik przekroczenia normy. 


In [ ]:
metadata_combined = pd.read_csv("cleaned_data/pd3_dataset_metadata.csv.xz")
data_combined = pd.read_csv("cleaned_data/pd3_dataset_data.csv.xz", index_col=0, low_memory=False)
data_combined.index = pd.to_datetime(data_combined.index, format='mixed')
data_combined
# Zadanie 5
from visualizations import plot_pm25_days_over_by_voivodeship_years

plot_pm25_days_over_by_voivodeship_years(
    meas_df=data_combined,
    metadata_df=metadata_combined,
    years=[2015, 2018, 2021, 2024],
    threshold=15
)
